In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,2021-02-09T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1161,176,1337,37184,...,NaN,5.0,NaN,NaN,142776.0,0.0,1587121.0,418792.0,ITG,ITG1
7388,2021-02-09T17:00:00,ITA,9,Toscana,43.769231,11.255889,701,113,814,9927,...,NaN,9.0,NaN,NaN,139404.0,190.0,2230318.0,133386.0,ITI,ITI1
7389,2021-02-09T17:00:00,ITA,10,Umbria,43.106758,12.388247,434,79,513,6660,...,"Si fa presente che 37 dei ricoveri NON UTI, so...",9.0,NaN,NaN,39049.0,0.0,626233.0,64879.0,ITI,ITI2
7390,2021-02-09T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,19,3,22,133,...,NaN,0.0,NaN,NaN,7870.0,0.0,70718.0,1546.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,2021-02-09T17:00:00,19,Sicilia,1161,176,1337,37184,38521,-411,744,...,NaN,NaN,142776,2005913,1033929.0,5.0,142776.0,0.0,1587121.0,418792.0
7388,2021-02-09T17:00:00,9,Toscana,701,113,814,9927,10741,22,453,...,NaN,NaN,139594,2363704,1255241.0,9.0,139404.0,190.0,2230318.0,133386.0
7389,2021-02-09T17:00:00,10,Umbria,434,79,513,6660,7173,271,375,...,NaN,NaN,39049,691112,294399.0,9.0,39049.0,0.0,626233.0,64879.0
7390,2021-02-09T17:00:00,2,Valle d'Aosta,19,3,22,133,155,-10,14,...,NaN,NaN,7870,72264,42169.0,0.0,7870.0,0.0,70718.0,1546.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-09', '2021-02-08')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-09T17:00:00,1,Piemonte,2040,148,2188,9883,12071,-220,619,...,NaN,NaN,229381,2215289,1140808.0,3.0,226555.0,2826.0,1936941.0,278348.0
1,2021-02-09T17:00:00,2,Valle d'Aosta,19,3,22,133,155,-10,14,...,NaN,NaN,7870,72264,42169.0,0.0,7870.0,0.0,70718.0,1546.0
2,2021-02-09T17:00:00,3,Lombardia,3553,363,3916,43123,47039,665,1625,...,NaN,NaN,552005,5923079,2906344.0,27.0,549315.0,2690.0,5719228.0,203851.0
3,2021-02-09T17:00:00,5,Veneto,1112,147,1259,25071,26330,-600,701,...,NaN,NaN,317979,4415169,1344750.0,11.0,316179.0,1800.0,3837128.0,578041.0
4,2021-02-09T17:00:00,6,Friuli Venezia Giulia,501,66,567,9979,10546,-196,271,...,NaN,NaN,70515,1204993,437096.0,5.0,67202.0,3313.0,1151547.0,53446.0
5,2021-02-09T17:00:00,7,Liguria,616,63,679,4051,4730,-64,257,...,NaN,NaN,72226,912708,393393.0,4.0,72226.0,0.0,852532.0,60176.0
6,2021-02-09T17:00:00,8,Emilia-Romagna,1971,182,2153,40731,42884,111,977,...,NaN,NaN,229157,3304742,1396803.0,7.0,229091.0,66.0,3104741.0,200001.0
7,2021-02-09T17:00:00,9,Toscana,701,113,814,9927,10741,22,453,...,NaN,NaN,139594,2363704,1255241.0,9.0,139404.0,190.0,2230318.0,133386.0
8,2021-02-09T17:00:00,10,Umbria,434,79,513,6660,7173,271,375,...,NaN,NaN,39049,691112,294399.0,9.0,39049.0,0.0,626233.0,64879.0
9,2021-02-09T17:00:00,11,Marche,542,74,616,6989,7605,-64,270,...,NaN,NaN,58582,729110,439935.0,3.0,58582.0,0.0,692593.0,36517.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-08T17:00:00,1,Piemonte,2036,148,2184,10107,12291,-31,483,...,NaN,NaN,228762,2196114,1134772.0,13.0,226090.0,2672.0,1929202.0,266912.0
1,2021-02-08T17:00:00,2,Valle d'Aosta,25,3,28,137,165,3,13,...,NaN,NaN,7856,71962,42078.0,0.0,7856.0,0.0,70510.0,1452.0
2,2021-02-08T17:00:00,3,Lombardia,3491,362,3853,42521,46374,-2466,895,...,NaN,NaN,550380,5893600,2899283.0,22.0,547948.0,2432.0,5699756.0,193844.0
3,2021-02-08T17:00:00,5,Veneto,1110,153,1263,25667,26930,-40,442,...,NaN,NaN,317278,4380776,1342044.0,6.0,315478.0,1800.0,3827806.0,552970.0
4,2021-02-08T17:00:00,6,Friuli Venezia Giulia,527,63,590,10152,10742,-45,132,...,NaN,NaN,70244,1198414,434761.0,9.0,67018.0,3226.0,1147226.0,51188.0
5,2021-02-08T17:00:00,7,Liguria,609,63,672,4122,4794,143,220,...,NaN,NaN,71969,906468,391081.0,3.0,71969.0,0.0,848493.0,57975.0
6,2021-02-08T17:00:00,8,Emilia-Romagna,2002,183,2185,40588,42773,-488,1273,...,NaN,NaN,228198,3275041,1392846.0,9.0,228131.0,67.0,3088607.0,186434.0
7,2021-02-08T17:00:00,9,Toscana,691,112,803,9916,10719,279,523,...,NaN,NaN,139141,2347868,1248041.0,10.0,138960.0,181.0,2221804.0,126064.0
8,2021-02-08T17:00:00,10,Umbria,423,77,500,6402,6902,118,190,...,NaN,NaN,38674,681597,292665.0,6.0,38674.0,0.0,621186.0,60411.0
9,2021-02-08T17:00:00,11,Marche,536,73,609,7060,7669,43,171,...,NaN,NaN,58312,724002,436633.0,3.0,58312.0,0.0,689309.0,34693.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-09T17:00:00,1,Piemonte,2040,148,2188,9883,12071,-220,619,...,NaN,NaN,229381,2215289,1140808.0,3.0,226555.0,2826.0,1936941.0,278348.0
1,2021-02-09T17:00:00,2,Valle d'Aosta,19,3,22,133,155,-10,14,...,NaN,NaN,7870,72264,42169.0,0.0,7870.0,0.0,70718.0,1546.0
2,2021-02-09T17:00:00,3,Lombardia,3553,363,3916,43123,47039,665,1625,...,NaN,NaN,552005,5923079,2906344.0,27.0,549315.0,2690.0,5719228.0,203851.0
3,2021-02-09T17:00:00,4,Trentino-Alto Adige,394,59,453,7775,8228,175,885,...,0.0,0.0,74595,1115162,350833.0,3.0,64272.0,10323.0,970346.0,144816.0
4,2021-02-09T17:00:00,5,Veneto,1112,147,1259,25071,26330,-600,701,...,NaN,NaN,317979,4415169,1344750.0,11.0,316179.0,1800.0,3837128.0,578041.0
5,2021-02-09T17:00:00,6,Friuli Venezia Giulia,501,66,567,9979,10546,-196,271,...,NaN,NaN,70515,1204993,437096.0,5.0,67202.0,3313.0,1151547.0,53446.0
6,2021-02-09T17:00:00,7,Liguria,616,63,679,4051,4730,-64,257,...,NaN,NaN,72226,912708,393393.0,4.0,72226.0,0.0,852532.0,60176.0
7,2021-02-09T17:00:00,8,Emilia-Romagna,1971,182,2153,40731,42884,111,977,...,NaN,NaN,229157,3304742,1396803.0,7.0,229091.0,66.0,3104741.0,200001.0
8,2021-02-09T17:00:00,9,Toscana,701,113,814,9927,10741,22,453,...,NaN,NaN,139594,2363704,1255241.0,9.0,139404.0,190.0,2230318.0,133386.0
9,2021-02-09T17:00:00,10,Umbria,434,79,513,6660,7173,271,375,...,NaN,NaN,39049,691112,294399.0,9.0,39049.0,0.0,626233.0,64879.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-02-08T17:00:00,1,Piemonte,2036,148,2184,10107,12291,-31,...,NaN,NaN,228762,2196114,1134772.0,13.0,226090.0,2672.0,1929202.0,266912.0
1,1,2021-02-08T17:00:00,2,Valle d'Aosta,25,3,28,137,165,3,...,NaN,NaN,7856,71962,42078.0,0.0,7856.0,0.0,70510.0,1452.0
2,2,2021-02-08T17:00:00,3,Lombardia,3491,362,3853,42521,46374,-2466,...,NaN,NaN,550380,5893600,2899283.0,22.0,547948.0,2432.0,5699756.0,193844.0
3,19,2021-02-08T17:00:00,4,Trentino-Alto Adige,391,58,449,7604,8053,88,...,0.0,0.0,73710,1101205,350238.0,3.0,64097.0,9613.0,967839.0,133366.0
4,3,2021-02-08T17:00:00,5,Veneto,1110,153,1263,25667,26930,-40,...,NaN,NaN,317278,4380776,1342044.0,6.0,315478.0,1800.0,3827806.0,552970.0
5,4,2021-02-08T17:00:00,6,Friuli Venezia Giulia,527,63,590,10152,10742,-45,...,NaN,NaN,70244,1198414,434761.0,9.0,67018.0,3226.0,1147226.0,51188.0
6,5,2021-02-08T17:00:00,7,Liguria,609,63,672,4122,4794,143,...,NaN,NaN,71969,906468,391081.0,3.0,71969.0,0.0,848493.0,57975.0
7,6,2021-02-08T17:00:00,8,Emilia-Romagna,2002,183,2185,40588,42773,-488,...,NaN,NaN,228198,3275041,1392846.0,9.0,228131.0,67.0,3088607.0,186434.0
8,7,2021-02-08T17:00:00,9,Toscana,691,112,803,9916,10719,279,...,NaN,NaN,139141,2347868,1248041.0,10.0,138960.0,181.0,2221804.0,126064.0
9,8,2021-02-08T17:00:00,10,Umbria,423,77,500,6402,6902,118,...,NaN,NaN,38674,681597,292665.0,6.0,38674.0,0.0,621186.0,60411.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-02-09T17:00:00,1,Piemonte,2040,148,2188,9883,12071,-220,619,...,NaN,229381,2215289,1140808.0,3.0,226555.0,2826.0,1936941.0,278348.0,4341375
1,2021-02-09T17:00:00,2,Valle d'Aosta,19,3,22,133,155,-10,14,...,NaN,7870,72264,42169.0,0.0,7870.0,0.0,70718.0,1546.0,125501
2,2021-02-09T17:00:00,3,Lombardia,3553,363,3916,43123,47039,665,1625,...,NaN,552005,5923079,2906344.0,27.0,549315.0,2690.0,5719228.0,203851.0,10103969
3,2021-02-09T17:00:00,4,Trentino-Alto Adige,394,59,453,7775,8228,175,885,...,0.0,74595,1115162,350833.0,3.0,64272.0,10323.0,970346.0,144816.0,1074819
4,2021-02-09T17:00:00,5,Veneto,1112,147,1259,25071,26330,-600,701,...,NaN,317979,4415169,1344750.0,11.0,316179.0,1800.0,3837128.0,578041.0,4907704
5,2021-02-09T17:00:00,6,Friuli Venezia Giulia,501,66,567,9979,10546,-196,271,...,NaN,70515,1204993,437096.0,5.0,67202.0,3313.0,1151547.0,53446.0,1211357
6,2021-02-09T17:00:00,7,Liguria,616,63,679,4051,4730,-64,257,...,NaN,72226,912708,393393.0,4.0,72226.0,0.0,852532.0,60176.0,1543127
7,2021-02-09T17:00:00,8,Emilia-Romagna,1971,182,2153,40731,42884,111,977,...,NaN,229157,3304742,1396803.0,7.0,229091.0,66.0,3104741.0,200001.0,4467118
8,2021-02-09T17:00:00,9,Toscana,701,113,814,9927,10741,22,453,...,NaN,139594,2363704,1255241.0,9.0,139404.0,190.0,2230318.0,133386.0,3722729
9,2021-02-09T17:00:00,10,Umbria,434,79,513,6660,7173,271,375,...,NaN,39049,691112,294399.0,9.0,39049.0,0.0,626233.0,64879.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-02-09T17:00:00,1,Piemonte,2040,148,2188,9883,12071,-220,619,...,229381,2215289,1140808.0,3.0,226555.0,2826.0,1936941.0,278348.0,4341375,575
1,2021-02-09T17:00:00,2,Valle d'Aosta,19,3,22,133,155,-10,14,...,7870,72264,42169.0,0.0,7870.0,0.0,70718.0,1546.0,125501,20
2,2021-02-09T17:00:00,3,Lombardia,3553,363,3916,43123,47039,665,1625,...,552005,5923079,2906344.0,27.0,549315.0,2690.0,5719228.0,203851.0,10103969,1036
3,2021-02-09T17:00:00,4,Trentino-Alto Adige,394,59,453,7775,8228,175,885,...,74595,1115162,350833.0,3.0,64272.0,10323.0,970346.0,144816.0,1074819,106
4,2021-02-09T17:00:00,5,Veneto,1112,147,1259,25071,26330,-600,701,...,317979,4415169,1344750.0,11.0,316179.0,1800.0,3837128.0,578041.0,4907704,1016
5,2021-02-09T17:00:00,6,Friuli Venezia Giulia,501,66,567,9979,10546,-196,271,...,70515,1204993,437096.0,5.0,67202.0,3313.0,1151547.0,53446.0,1211357,180
6,2021-02-09T17:00:00,7,Liguria,616,63,679,4051,4730,-64,257,...,72226,912708,393393.0,4.0,72226.0,0.0,852532.0,60176.0,1543127,209
7,2021-02-09T17:00:00,8,Emilia-Romagna,1971,182,2153,40731,42884,111,977,...,229157,3304742,1396803.0,7.0,229091.0,66.0,3104741.0,200001.0,4467118,634
8,2021-02-09T17:00:00,9,Toscana,701,113,814,9927,10741,22,453,...,139594,2363704,1255241.0,9.0,139404.0,190.0,2230318.0,133386.0,3722729,523
9,2021-02-09T17:00:00,10,Umbria,434,79,513,6660,7173,271,375,...,39049,691112,294399.0,9.0,39049.0,0.0,626233.0,64879.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-09T17:00:00,3,Lombardia,3553,363,3916,43123,47039,665,1625,...,35.04,602,730,29479,7061.0,55,905,5.51,0.01608,5.46
1,2021-02-09T17:00:00,15,Campania,1540,112,1652,63681,65333,-375,1274,...,22.18,-375,85,13691,7623.0,24,1625,9.31,0.02202,4.05
2,2021-02-09T17:00:00,8,Emilia-Romagna,1971,182,2153,40731,42884,111,977,...,28.71,143,-296,29701,3957.0,45,803,3.29,0.02187,5.13
3,2021-02-09T17:00:00,4,Trentino-Alto Adige,394,59,453,7775,8228,175,885,...,55.66,171,564,13957,595.0,9,701,6.34,0.08234,6.94
4,2021-02-09T17:00:00,12,Lazio,2257,268,2525,44521,47046,-2504,847,...,31.64,-2509,65,29609,19525.0,33,3318,2.86,0.01444,3.65
5,2021-02-09T17:00:00,19,Sicilia,1161,176,1337,37184,38521,-411,744,...,29.93,-375,266,21948,5420.0,24,1131,3.39,0.01497,2.87
6,2021-02-09T17:00:00,5,Veneto,1112,147,1259,25071,26330,-600,701,...,14.47,-596,259,34393,2706.0,65,1236,2.04,0.01428,6.48
7,2021-02-09T17:00:00,16,Puglia,1415,165,1580,46980,48560,-1725,681,...,44.72,-1691,304,9262,4323.0,41,2365,7.35,0.01699,3.25
8,2021-02-09T17:00:00,1,Piemonte,2040,148,2188,9883,12071,-220,619,...,25.74,-224,136,19175,6036.0,35,804,3.23,0.01426,5.28
9,2021-02-09T17:00:00,9,Toscana,701,113,814,9927,10741,22,453,...,21.61,11,-70,15836,7200.0,27,404,2.86,0.01217,3.75


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,79,38,117,67.52
1,Trentino-Alto Adige,59,47,106,55.66
2,Marche,74,69,143,51.75
3,Puglia,165,204,369,44.72
4,Friuli Venezia Giulia,66,114,180,36.67
5,Lombardia,363,673,1036,35.04
6,Abruzzo,51,97,148,34.46
7,Lazio,268,579,847,31.64
8,Liguria,63,146,209,30.14
9,Sicilia,176,412,588,29.93


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,2021-02-07T17:00:00,22,P.A. Trento,136,26,162,2514,2676,52,175,...,NaN,NaN,28908,548473,163600.0,0.0,25798.0,3110.0,522367.0,26106.0
700,2021-02-08T17:00:00,21,P.A. Bolzano,250,30,280,5176,5456,167,255,...,NaN,NaN,44736,551854,186416.0,0.0,38260.0,6476.0,444834.0,107020.0
701,2021-02-08T17:00:00,22,P.A. Trento,141,28,169,2428,2597,-79,66,...,NaN,NaN,28974,549351,163822.0,3.0,25837.0,3137.0,523005.0,26346.0
702,2021-02-09T17:00:00,21,P.A. Bolzano,250,30,280,5376,5656,200,721,...,NaN,NaN,45457,563445,186732.0,0.0,38416.0,7041.0,446377.0,117068.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,2021-02-05T17:00:00,4,Trentino-Alto Adige,397,67,464,8424,8888,555,889,...,6.0,11594.0,1106.0,551.0,14.44,5.22,7.67,80.38,0.00,6.66
348,2021-02-06T17:00:00,4,Trentino-Alto Adige,373,62,435,7181,7616,-1272,1080,...,6.0,12681.0,1118.0,-1243.0,14.25,5.71,8.52,96.60,-7.46,-14.31
349,2021-02-07T17:00:00,4,Trentino-Alto Adige,378,61,439,7526,7965,349,645,...,10.0,7678.0,1001.0,345.0,13.90,5.51,8.40,64.44,-1.61,4.58
350,2021-02-08T17:00:00,4,Trentino-Alto Adige,391,58,449,7604,8053,88,321,...,10.0,3595.0,566.0,78.0,12.92,5.58,8.93,56.71,-4.92,1.10


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
